# Mock API Client Demo
This notebook exercises the folder-backed mock API server.

Prereqs:
- Start the server in a separate terminal:

```bash
python mock_api_server.py \
  --key-table "D:/real_data_key/filtered_rows.xlsx" \
  --search-root "D:/test_data" \
  --tail-len 5 --any-depth
```

- Install deps in this Python env: `pip install requests pandas openpyxl pillow`


In [ ]:
import json, base64, io, requests
from pathlib import Path
from PIL import Image

ENDPOINT = "http://127.0.0.1:5001/images"
FILENAME = "N20231004THA00001"  # change to an existing '파일명' entry
SAVE_TO_FOLDER = True  # ask server to persist the TIFF


## POST request with minimal payload (filename only)
Server uses 'filename' to match, and pulls metadata (사업자등록번호, 수신일자, 업종명) from the key table if available.


In [ ]:
payload = {
    'filename': FILENAME,
    'save_to_folder': str(SAVE_TO_FOLDER).lower(),
}
r = requests.post(ENDPOINT, json=payload, timeout=30)
r.raise_for_status()
data = r.json()
print(json.dumps(data, ensure_ascii=False, indent=2))


## Decode returned base64 and display image inline


In [ ]:
b64 = data.get('image_b64')
if b64:
    raw = base64.b64decode(b64)
    img = Image.open(io.BytesIO(raw))
    display(img)
else:
    print('No image_b64 in response')


## GET request variant (useful for quick checks)


In [ ]:
params = {
    'filename': FILENAME,
    'save_to_folder': 'true',
}
r = requests.get(ENDPOINT, params=params, timeout=30)
r.raise_for_status()
data2 = r.json()
print(json.dumps(data2, ensure_ascii=False, indent=2))


## Build payload from key table (optional)
If needed, construct additional fields for the request by reading a key table row matching '파일명'.


In [ ]:
import pandas as pd
KEY_TABLE = Path('D:/real_data_key/filtered_rows.xlsx')  # adjust as needed
FILE_NAME_COLUMN = '파일명'
df = pd.read_excel(KEY_TABLE) if KEY_TABLE.suffix.lower() in ('.xlsx', '.xls') else pd.read_csv(KEY_TABLE)
row = df.loc[df[FILE_NAME_COLUMN].astype(str).str.strip() == FILENAME]
row = row.iloc[0].to_dict() if not row.empty else {}
extra = {}
# Map request parameters to key table columns
# e.g., 'comany_registration': '사업자등록번호', 'reception_date': '수신일자', 'company_name': '업종명'
mappings = {
    'comany_registration': '사업자등록번호',
    'reception_date': '수신일자',
    'company_name': '업종명',
}
for req_key, col in mappings.items():
    extra[req_key] = row.get(col) if row else None
payload2 = {'filename': FILENAME, 'save_to_folder': 'false', **extra}
r = requests.post(ENDPOINT, json=payload2, timeout=30)
r.raise_for_status()
data3 = r.json()
print(json.dumps(data3, ensure_ascii=False, indent=2))
